### Original_textで学習回してみるnotebook
* いけました

In [1]:
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from transformers import AdamW

from config import *
from bert_utils import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

****** SEED fixed : 256 ******




In [4]:
settings = pd.Series(dict(
    folds=5,
    model_name="cl-tohoku/bert-base-japanese",
    train_batch_size=4,
    valid_batch_size=4,
    #max_length=128,
    max_length=1024,
    num_classes=2,
    n_accumulate=1,
    use_amp=True,
    epochs=2,
    output_path="./output/org_text_tmp/",
    model_custom_header="concatenate-4",
    dropout=0.2,
    learning_rate=1e-5,
    weight_decay=1e-6,
    scheduler_name="TEST"
))
log = open(settings.output_path + "/train.log", "w", buffering=1)

In [5]:
# preprocess --
train_df = pd.read_feather("input/dataset_with_original_text/train_with_original_text.feather")
train_df["clean_text"] = train_df["original_text"].map(lambda x: original_text_preprocess(x))

# make folds --
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
split = skf.split(train_df, train_df[label_name])
train_df = make_folds(split, train_df, label_name=label_name)

# define tokenizer --
tokenizer = define_tokenizer(settings.model_name)

In [6]:
for fold in range(0, settings.folds):

    # Create DataLoader --
    train_loader, valid_loader = prepare_loaders(
        df=train_df,
        tokenizer=tokenizer,
        fold=fold,
        trn_batch_size=settings.train_batch_size,
        val_batch_size=settings.valid_batch_size,
        max_length=settings.max_length,
        num_classes=settings.num_classes,
        text_col="clean_text"
    )

    # Model construct --
    model = HateSpeechModel(
        model_name=settings.model_name,
        num_classes=settings.num_classes,
        custom_header=settings.model_custom_header,
        dropout=settings.dropout,
        )

    # Define Optimizer and Scheduler --
    optimizer = AdamW(model.parameters(), lr=settings.learning_rate, weight_decay=settings.weight_decay)
    scheduler = fetch_scheduler(optimizer=optimizer, scheduler=settings.scheduler_name)

    model.to(device)
    model, history = run_training(
        model, train_loader, valid_loader, 
        optimizer, scheduler, settings.n_accumulate, device, settings.use_amp, 
        settings.epochs, fold, settings.output_path,
        log, save_checkpoint=False
    )

    del model, history, train_loader, valid_loader
    _ = gc.collect()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


concatenate-4
*** *** NOT implemented *** *** 
        --> CosineAnnealingLR *** *** 
[INFO] Using GPU : NVIDIA GeForce RTX 3090



  0%|          | 0/1051 [00:00<?, ?it/s]


RuntimeError: The expanded size of the tensor (1024) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [4, 1024].  Tensor sizes: [1, 512]